## Introduksjon 

Denne leksjonen vil dekke: 
- Hva funksjonskalling er og bruksområdene 
- Hvordan lage et funksjonskall ved bruk av OpenAI 
- Hvordan integrere et funksjonskall i en applikasjon 

## Læringsmål 

Etter å ha fullført denne leksjonen vil du vite hvordan og forstå: 

-  Formålet med å bruke funksjonskalling 
- Sette opp funksjonskall ved bruk av OpenAI-tjenesten 
- Designe effektive funksjonskall for applikasjonens bruksområde 


## Forstå funksjonskall

For denne leksjonen ønsker vi å bygge en funksjon for vår utdanningsstartup som lar brukere bruke en chatbot for å finne tekniske kurs. Vi vil anbefale kurs som passer deres ferdighetsnivå, nåværende rolle og teknologi av interesse.

For å fullføre dette vil vi bruke en kombinasjon av:
 - `OpenAI` for å lage en chatteopplevelse for brukeren
 - `Microsoft Learn Catalog API` for å hjelpe brukere med å finne kurs basert på brukerens forespørsel
 - `Function Calling` for å ta brukerens spørsmål og sende det til en funksjon for å gjøre API-forespørselen.

For å komme i gang, la oss se på hvorfor vi ønsker å bruke funksjonskall i utgangspunktet:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # få et nytt svar fra GPT hvor det kan se funksjonssvaret


print(second_response.choices[0].message)


### Hvorfor Funksjonskalling

Hvis du har fullført noen annen leksjon i dette kurset, forstår du sannsynligvis kraften i å bruke store språkmodeller (LLMs). Forhåpentligvis kan du også se noen av begrensningene deres.

Funksjonskalling er en funksjon i OpenAI-tjenesten designet for å løse følgende utfordringer:

Uensartet responsformat:
- Før funksjonskalling var svar fra en stor språkmodell ustrukturert og inkonsekvent. Utviklere måtte skrive kompleks valideringskode for å håndtere hver variasjon i utdataene.

Begrenset integrasjon med ekstern data:
- Før denne funksjonen var det vanskelig å innlemme data fra andre deler av en applikasjon i en chat-kontekst.

Ved å standardisere responsformater og muliggjøre sømløs integrasjon med ekstern data, forenkler funksjonskalling utviklingen og reduserer behovet for ekstra valideringslogikk.

Brukere kunne ikke få svar som "Hva er været i Stockholm akkurat nå?". Dette skyldes at modellene var begrenset til tidspunktet dataene ble trent på.

La oss se på eksempelet nedenfor som illustrerer dette problemet:

La oss si at vi ønsker å lage en database med studentdata slik at vi kan foreslå riktig kurs til dem. Nedenfor har vi to beskrivelser av studenter som er veldig like i dataene de inneholder.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vi ønsker å sende dette til en LLM for å analysere dataene. Dette kan senere brukes i applikasjonen vår for å sende dette til en API eller lagre det i en database.

La oss lage to identiske prompts der vi instruerer LLM om hvilken informasjon vi er interessert i:


Vi ønsker å sende dette til en LLM for å analysere de delene som er viktige for produktet vårt. Slik at vi kan lage to identiske prompt for å instruere LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Etter å ha laget disse to promptene, sender vi dem til LLM ved å bruke `openai.ChatCompletion`. Vi lagrer prompten i variabelen `messages` og tildeler rollen til `user`. Dette er for å etterligne en melding fra en bruker som skrives til en chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nå kan vi sende begge forespørslene til LLM og undersøke svaret vi mottar.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Selv om promptene er de samme og beskrivelsene ligner, kan vi få forskjellige formater på `Grades`-egenskapen.

Hvis du kjører cellen ovenfor flere ganger, kan formatet være `3.7` eller `3.7 GPA`.

Dette er fordi LLM tar ustrukturert data i form av den skrevne prompten og returnerer også ustrukturert data. Vi trenger å ha et strukturert format slik at vi vet hva vi kan forvente når vi lagrer eller bruker disse dataene.

Ved å bruke funksjonskall kan vi sørge for at vi mottar strukturerte data tilbake. Når vi bruker funksjonskall, kaller eller kjører ikke LLM faktisk noen funksjoner. I stedet lager vi en struktur for at LLM skal følge i sine svar. Vi bruker deretter disse strukturerte svarene for å vite hvilken funksjon vi skal kjøre i våre applikasjoner.


![Funksjonskall Flytdiagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.no.png)


Vi kan deretter ta det som returneres fra funksjonen og sende dette tilbake til LLM. LLM vil deretter svare med naturlig språk for å besvare brukerens spørsmål.


### Bruksområder for bruk av funksjonskall

**Kalle eksterne verktøy**  
Chatboter er gode til å gi svar på spørsmål fra brukere. Ved å bruke funksjonskall kan chatbotene bruke meldinger fra brukere til å utføre visse oppgaver. For eksempel kan en student be chatboten om å "Sende e-post til min instruktør og si at jeg trenger mer hjelp med dette emnet". Dette kan gjøre et funksjonskall til `send_email(to: string, body: string)`

**Opprette API- eller databaseforespørsler**  
Brukere kan finne informasjon ved å bruke naturlig språk som blir konvertert til en formatert forespørsel eller API-forespørsel. Et eksempel på dette kan være en lærer som spør "Hvem er studentene som fullførte den siste oppgaven" som kan kalle en funksjon kalt `get_completed(student_name: string, assignment: int, current_status: string)`

**Opprette strukturert data**  
Brukere kan ta et tekstavsnitt eller CSV og bruke LLM til å hente ut viktig informasjon fra det. For eksempel kan en student konvertere en Wikipedia-artikkel om fredsavtaler for å lage AI-flashkort. Dette kan gjøres ved å bruke en funksjon kalt `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Lage ditt første funksjonskall

Prosessen med å lage et funksjonskall inkluderer 3 hovedtrinn:
1. Kalle Chat Completions API med en liste over funksjonene dine og en brukermelding
2. Lese modellens svar for å utføre en handling, dvs. kjøre en funksjon eller API-kall
3. Gjøre et nytt kall til Chat Completions API med svaret fra funksjonen din for å bruke den informasjonen til å lage et svar til brukeren.


![Flyt av et Funksjonskall](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.no.png)


### Elementer i et funksjonskall 

#### Brukerens inndata 

Det første steget er å lage en brukermelding. Denne kan tildeles dynamisk ved å ta verdien fra et tekstfelt, eller du kan tildele en verdi her. Hvis dette er første gang du jobber med Chat Completions API, må vi definere `role` og `content` i meldingen. 

`role` kan være enten `system` (opprette regler), `assistant` (modellen) eller `user` (sluttbrukeren). For funksjonskall vil vi tildele dette som `user` og et eksempelspørsmål. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Lage funksjoner.

Neste skal vi definere en funksjon og parameterne til den funksjonen. Vi vil bruke bare én funksjon her kalt `search_courses`, men du kan lage flere funksjoner.

**Viktig**: Funksjoner er inkludert i systemmeldingen til LLM og vil bli inkludert i mengden tilgjengelige tokens du har tilgjengelig.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definisjoner** 

Funksjonsdefinisjonsstrukturen har flere nivåer, hver med sine egne egenskaper. Her er en oversikt over den nestede strukturen:

**Funksjonsegenskaper på øverste nivå:**

`name` - Navnet på funksjonen vi ønsker å få kalt. 

`description` - Dette er beskrivelsen av hvordan funksjonen fungerer. Her er det viktig å være spesifikk og tydelig 

`parameters` - En liste over verdier og format som du ønsker at modellen skal produsere i sitt svar 

**Egenskaper for parameterobjektet:**

`type` - Datatypen til parameterobjektet (vanligvis "object")

`properties` - Liste over de spesifikke verdiene som modellen vil bruke i sitt svar 

**Egenskaper for individuelle parametere:**

`name` - Implisitt definert av egenskapsnøkkelen (f.eks. "role", "product", "level")

`type` - Datatypen til denne spesifikke parameteren (f.eks. "string", "number", "boolean") 

`description` - Beskrivelse av den spesifikke parameteren 

**Valgfrie egenskaper:**

`required` - En liste som angir hvilke parametere som er nødvendige for at funksjonskallet skal fullføres 


### Lage funksjonskallet  
Etter å ha definert en funksjon, må vi nå inkludere den i kallet til Chat Completion API. Vi gjør dette ved å legge til `functions` i forespørselen. I dette tilfellet `functions=functions`.  

Det finnes også et alternativ for å sette `function_call` til `auto`. Dette betyr at vi lar LLM bestemme hvilken funksjon som skal kalles basert på brukermeldingen i stedet for å tildele det selv.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

La oss nå se på svaret og hvordan det er formatert:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Du kan se at navnet på funksjonen blir kalt, og ut fra brukerens melding klarte LLM å finne dataene som passer til argumentene til funksjonen.


## 3.Integrering av funksjonskall i en applikasjon. 


Etter at vi har testet det formaterte svaret fra LLM, kan vi nå integrere dette i en applikasjon. 

### Håndtering av flyten 

For å integrere dette i applikasjonen vår, la oss ta følgende steg: 

Først, la oss gjøre kallet til OpenAI-tjenestene og lagre meldingen i en variabel kalt `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nå skal vi definere funksjonen som vil kalle Microsoft Learn API for å hente en liste over kurs:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Som en beste praksis vil vi deretter se om modellen ønsker å kalle en funksjon. Etter det vil vi opprette en av de tilgjengelige funksjonene og matche den med funksjonen som blir kalt.  
Vi vil deretter ta argumentene til funksjonen og kartlegge dem til argumenter fra LLM.

Til slutt vil vi legge til funksjonskallmeldingen og verdiene som ble returnert av `search_courses`-meldingen. Dette gir LLM all informasjonen den trenger for å svare brukeren med naturlig språk.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nå vil vi sende den oppdaterte meldingen til LLM slik at vi kan motta et svar på naturlig språk i stedet for et API JSON-formatert svar.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodeutfordring 

Flott arbeid! For å fortsette læringen din om OpenAI Function Calling kan du bygge: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Flere parametere for funksjonen som kan hjelpe lærere med å finne flere kurs. Du kan finne tilgjengelige API-parametere her: 
 - Lag et annet funksjonskall som tar mer informasjon fra læreren, som deres morsmål 
 - Lag feilhåndtering når funksjonskallet og/eller API-kallet ikke returnerer noen passende kurs 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på originalspråket skal anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
